In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Set your data directory
data_dir = '/kaggle/input/brahmi-dataset/data/'
batch_size = 32
image_size = (128, 128)
num_classes = 344  # Update with the actual number of classes

# Enhanced data augmentation for training
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,  # Increased rotation range
    width_shift_range=0.1,  # Increased width shift range
    height_shift_range=0.1,  # Increased height shift range
    brightness_range=[0.9, 1.1],  # Expanded brightness range
    zoom_range=0.1,  # Increased zoom range
    horizontal_flip=False,  # Allowing horizontal flipping
    vertical_flip=False,  # Allowing vertical flipping
    shear_range=0.1,  # Increased shear range
    fill_mode='nearest',
    validation_split=0.15
)

In [3]:
# Create generators for training and validation
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    subset='training'  # Specify subset as 'training' for the training generator
)

val_generator = datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    subset='validation'  # Specify subset as 'validation' for the validation generator
)

Found 6698 images belonging to 344 classes.
Found 1182 images belonging to 344 classes.


In [4]:
# Deeper model architecture with pooling and regularization
model = tf.keras.Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.3),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.3),

    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.5),

    Flatten(),

    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),

    Dense(num_classes, activation='softmax')
])

# Compile the model using Adam optimizer with momentum
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 64)      1792      
                                                                 
 batch_normalization (Batch  (None, 126, 126, 64)      256       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 63, 63, 64)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 63, 63, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        36928     
                                                                 
 batch_normalization_1 (Bat  (None, 61, 61, 64)        2

In [6]:
# Function to adjust learning rate during training
def lr_scheduler(epoch, lr):
    if epoch % 5 == 0 and epoch > 0:
        lr = lr * 0.75  # Reduce learning rate by 30% every 5 epochs
    return lr

# Set up the step decay learning rate scheduler callback
lr_schedule = LearningRateScheduler(lr_scheduler)

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)  # Adjust patience

In [7]:
# Using ImageDataGenerator's flow method during training and validation
model.fit(
    train_generator,
    epochs=100,
    validation_data=val_generator,
    callbacks=[lr_schedule, early_stopping]
)

Epoch 1/100


2024-01-15 17:13:05.549677: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


210/210 [==============================] - 84s 370ms/step - loss: 5.9549 - accuracy: 0.0245 - val_loss: 9.1788 - val_accuracy: 0.0127 - lr: 0.0010
Epoch 2/100
210/210 [==============================] - 46s 221ms/step - loss: 5.1684 - accuracy: 0.0578 - val_loss: 13.3567 - val_accuracy: 0.0085 - lr: 0.0010
Epoch 3/100
210/210 [==============================] - 47s 222ms/step - loss: 4.6983 - accuracy: 0.0944 - val_loss: 5.0251 - val_accuracy: 0.0584 - lr: 0.0010
Epoch 4/100
210/210 [==============================] - 47s 223ms/step - loss: 4.1221 - accuracy: 0.1500 - val_loss: 6.9993 - val_accuracy: 0.0228 - lr: 0.0010
Epoch 5/100
210/210 [==============================] - 47s 222ms/step - loss: 3.5958 - accuracy: 0.2154 - val_loss: 3.9635 - val_accuracy: 0.1531 - lr: 0.0010
Epoch 6/100
210/210 [==============================] - 48s 227ms/step - loss: 3.0922 - accuracy: 0.2916 - val_loss: 4.2578 - val_accuracy: 0.1277 - lr: 7.5000e-04
Epoch 7/100
210/210 [==============================] 

In [8]:
# The evaluation is done on the validation set
test_loss, test_acc = model.evaluate(val_generator)

37/37 [==============================] - 7s 187ms/step - loss: 0.3855 - accuracy: 0.9002


In [9]:
model.save('brahmi_recognition_model 9049%.h5')

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
